# Using TensorFlow on Spark Exploration

## Test Environment:
* Python = 3.9.7
* TensorFlow = 2.7.0
* PySpark = 3.1.2

## Inferencing Images with Trained Model
### Importing packages

In [ ]:
import tensorflow as tf
from pyspark import SparkContext
sc = SparkContext()

### Broadcasting current model to each cluster
This block of codes broadcasts the trained model to each cluster using sc.broadcast

In [ ]:
with tf.gfile.FastGFile( 'image_path') as f:
    model_data = f.read()
    model_data_bc = sc.broadcast(model_data)

### Applying the model to the images in each node
This block of codes runs the model on the ditributed nodes

In [ ]:
# Creates a new TensorFlow graph   
def apply_batch(image_url):
  with tf.Graph().as_default() as g:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(model_data_bc.value)
    tf.import_graph_def(graph_def, name='')

# Loads the image data from the URL:
  image_data = urllib.request.urlopen(img_url, timeout=1.0).read()

# Runs the model
  with tf.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
  return predictions